In [19]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [12]:
#Beispieldatensatz: Census Income Data Set vom UC Irvine Machine Learning Repository
df_census = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
df_census.columns =['age', 'work class', 'fnlwgt', 'education', 'education-num', 'marital-status',
                    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df_census = df_census.drop(['education'], axis=1)
df_census = pd.get_dummies(df_census)
df_census = df_census.drop('income_ <=50K', axis=1)
#Problemstellung: Will die Person mehr oder weniger Geld als 50k verdienen?
X = df_census.iloc[:, :-1]
y = df_census.iloc[:, -1]

In [34]:
class CustomGradientBoostingClassifier:
    
    def __init__(self, n_estimators, learning_rate, max_depth):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
     
    def fit(self, X, y): 
        
        F0 = np.log(y.mean()/(1-y.mean())) 
        self.F0 = np.full(len(y), F0)  
        Fm = self.F0.copy()
        
        for _ in range(self.n_estimators):  
            p = np.exp(Fm) / (1 + np.exp(Fm))  
            r = y - p  
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=0) 
            tree.fit(X, r)
            terminal_idx = tree.apply(X)  #array

             
            for j in np.unique(terminal_idx): 
              mask = terminal_idx == j

            
              neg_grad = r[mask].sum()       
              hessian = (p[mask]*(1-p[mask])).sum() 
              gamma = neg_grad / hessian

              
              Fm[mask] += self.learning_rate * gamma 

              
              tree.tree_.value[j, 0, 0] = gamma  

            self.trees.append(tree)
            
    def predict_proba(self, X):
        
        Fm = self.F0     
        
        for i in range(self.n_estimators):
            Fm += self.learning_rate * self.trees[i].predict(X) 
            
        return np.exp(Fm) / (1 + np.exp(Fm))
def predict(self, X, threshold=0.5):
       proba = self.predict_proba(X)
       return (proba >= threshold).astype(int)

In [39]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss

custom_gbm = CustomGradientBoostingClassifier(
    n_estimators=1, 
    learning_rate=0.1, 
    max_depth=3
)
custom_gbm.fit(X, y)
custom_log_loss = log_loss(y, custom_gbm.predict_proba(X))
print(f"Custom GBM Log-Loss:{custom_log_loss:.4f}")

gbm = GradientBoostingClassifier(
    n_estimators=1, 
    learning_rate=0.1, 
    max_depth=3
)
gbm.fit(X, y)
gbm_log_loss = log_loss(y, gbm.predict_proba(X))
print(f"Scikit-learn GBM Log-Loss:{gbm_log_loss:.4f}")
custom_gbm.predict(self, X=X, threshold=0.5)

Custom GBM Log-Loss:0.5162
Scikit-learn GBM Log-Loss:0.5162


AttributeError: 'CustomGradientBoostingClassifier' object has no attribute 'predict'